In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report, accuracy_score

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [3]:
# Checking the data for any missing entries
check_nan = lending_df.isnull().values.any()
check_nan

False

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]

# Separate the X variable, the features
X = lending_df.drop("loan_status", axis = 1)

In [5]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [6]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [7]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

A value of 1 means the loan has a high risk of defaulting. Based on the balance of the loan status, the number of loans that are at risk of defaulting in the dataset is 2500.

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

print("Size of training set:", X_train.shape)
print("Size of testing set:", X_test.shape)

Size of training set: (58152, 7)
Size of testing set: (19384, 7)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver = 'lbfgs',
                                max_iter = 200,
                                random_state = 1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
prediction_results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
prediction_results.head(15)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9520479254722232

In [12]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, predictions)
print(test_matrix)

[[18663   102]
 [   56   563]]


In [13]:
# Print the classification report for the model
target_names = ['healthy', 'high-risk']
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

     healthy       1.00      0.99      1.00     18765
   high-risk       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
The balanced accuracy score was calculated for the model to be 95%. The balanced accuracy in binary classification problems is used to deal with imbalanced dataset. In the case of our model, as per the confusion matrix, the dataset has a much higher number of healthy loan status data points than high-risk loan data points, thus we use the balanced accuracy score from the sklearn library vs the accuracy score. The balanced accuracy score is defined as the average of recall obtained on each class (0 or 1 in the case of our dataset). The higher the value of the score, the better our model accuracy is.

Based on the classification report, the model has a high precision for predicting the healthy loans. Precision is a measure of what proportion of positive identifications was correct. A model that has no false positives has a precision of 1.0. In the case of the regression model, the precision is 1.0, which means that when it predicts the loan_status to be healthy, it is correct 100% of the time. On the same note, when our model predicts the loan status to be high-risk, it is correct 85% of the time. 

Recall attempts to answer the question of what proportion of actual positives was identified correctly. A model that produces no false negatives has a recall of 1.0. In the case of the regression model, both healthy and high-risk recall scores are high. It is able to correctly identify 91% of high-risk loans, and 99% of healthy loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# imblearn install required to carry out the requirements of this assignment
!pip install imblearn


[notice] A new release of pip available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X, y)

In [17]:
# Count the distinct values of the resampled labels data
y_res.value_counts()

0    75036
1    75036
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [18]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_classifier = LogisticRegression(solver = 'lbfgs',
                                max_iter = 200,
                                random_state = 1)

# Split the datasets into training and testing dataset
X_train_ros, X_test_ros, y_train_ros, y_test_ros = train_test_split(X_res, y_res, random_state = 1)

print("Size of training set:", X_train_ros.shape)
print("Size of testing set:", X_test_ros.shape)

# Fit the model using the resampled training data
resampled_classifier.fit(X_train_ros, y_train_ros)

# Make a prediction using the testing data
predictions_ros = resampled_classifier.predict(X_test_ros)
prediction_results_ros = pd.DataFrame({"Prediction": predictions_ros, "Actual": y_test_ros}).reset_index(drop=True)
prediction_results_ros.head(15)

Size of training set: (112554, 7)
Size of testing set: (37518, 7)


,Prediction,Actual
0,0,0
1,1,1
2,1,1
3,0,0
4,0,0
5,1,1
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
# Print the balanced_accuracy score of the model 
accuracy_score(y_test_ros, predictions_ros)

0.9950157257849566

In [20]:
# Generate a confusion matrix for the model
test_matrix_ros = confusion_matrix(y_test_ros, predictions_ros)
print(test_matrix_ros)

[[18810    95]
 [   92 18521]]


In [21]:
# Print the classification report for the model
print(classification_report(y_test_ros, predictions_ros, target_names=target_names))

              precision    recall  f1-score   support

     healthy       1.00      0.99      1.00     18905
   high-risk       0.99      1.00      0.99     18613

    accuracy                           1.00     37518
   macro avg       1.00      1.00      1.00     37518
weighted avg       1.00      1.00      1.00     37518



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
When using the oversampled data, the logistic regression model performs much better, having an almost 100% precision and recall for both the healthy and high-risk loan data points. The accuracy score is also higher, at 99.5%. Although the accuracy of the healthy loan data points has not changed, now that we had more high-risk data points, the recall and precision improved significantly.